In [224]:
# Dataset load
import pandas as pd
data = pd.read_csv('dataset/haberman.csv')
# Dataset split into trainset testset 5:1 ratio
from sklearn.model_selection import train_test_split
trainset, testset = train_test_split(data,test_size=0.2,random_state=1234, shuffle=True)


In [225]:
# Splitting into x_train, y_train, x_test, y_test 
# Changing class value from 1, 2 to 0, 1
x_train, y_train = [], []

for i in range(len(trainset)):
    x_train.append(list(trainset.iloc[i, 0:3]))
    y_train.append(list(trainset.iloc[i, 3:4])[0])

x_test, y_test = [], []

for i in range(len(testset)):
    x_test.append(list(testset.iloc[i, 0:3]))
    y_test.append(list(testset.iloc[i, 3:4])[0])
    
for i in range(len(y_train)):
    if y_train[i] == 2:
        y_train[i] = 1
    else:
        y_train[i] = 0
for i in range(len(y_test)):
    if y_test[i] == 2:
        y_test[i] = 1
    else:
        y_test[i] = 0

In [226]:
# Data scaling with MaxAbsScaler
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

1. Data re uploading classifier with pennylane

In [227]:
import pennylane as qml
from pennylane import numpy as np
import torch
import torch.optim as optim

Define single qubit data reuploading circuit, both circuits below are same but difference is that one returns probs the other returns state

In [228]:

devc = qml.device("default.qubit", wires=1, shots=None)
@qml.qnode(devc, interface="torch")
def compact_circ2(params, x, num_layer):
    phi = []
    num_input = len(x)
    #alpha = params[0:2]
    weight = params[2:2+num_input*num_layer].reshape(num_layer,num_input)
    bias = params[2+num_input*num_layer:len(params)].reshape(num_layer,3)
    for i in range(num_layer):
        phi.append(torch.tanh(torch.inner(weight[i], x)))
        
    if num_layer == 1:
        qml.Rot(phi[0]+np.pi*torch.tanh(bias[0][0]), phi[0]+np.pi*torch.tanh(bias[0][1]), phi[0]+np.pi*torch.tanh(bias[0][2]), wires=0)
    else:
        qml.RZ(phi[0]+np.pi*torch.tanh(bias[0][2]), wires=0)
        qml.RY(phi[0]+np.pi*torch.tanh(bias[0][1]), wires=0)
        for j in range(num_layer-1):
            qml.RZ(phi[j]+np.pi*torch.tanh(bias[j][2])+phi[j+1]+np.pi*torch.tanh(bias[j+1][0]), wires=0)
            qml.RY(phi[j]+np.pi*torch.tanh(bias[j][1]), wires=0)
            
        qml.RZ(phi[num_layer-1]+np.pi*torch.tanh(bias[num_layer-1][2]), wires=0)
    return qml.probs(0)

In [229]:
devc = qml.device("default.qubit", wires=1, shots=None)
@qml.qnode(devc, interface="torch")
def compact_circ(params, x, num_layer):
    phi = []
    num_input = len(x)
    #alpha = params[0:2]
    weight = params[2:2+num_input*num_layer].reshape(num_layer,num_input)
    bias = params[2+num_input*num_layer:len(params)].reshape(num_layer,3)
    for i in range(num_layer):
        phi.append(torch.tanh(torch.inner(weight[i], x)))
        
    if num_layer == 1:
        qml.Rot(phi[0]+np.pi*torch.tanh(bias[0][0]), phi[0]+np.pi*torch.tanh(bias[0][1]), phi[0]+np.pi*torch.tanh(bias[0][2]), wires=0)
    else:
        qml.RZ(phi[0]+np.pi*torch.tanh(bias[0][2]), wires=0)
        qml.RY(phi[0]+np.pi*torch.tanh(bias[0][1]), wires=0)
        for j in range(num_layer-1):
            qml.RZ(phi[j]+np.pi*torch.tanh(bias[j][2])+phi[j+1]+np.pi*torch.tanh(bias[j+1][0]), wires=0)
            qml.RY(phi[j]+np.pi*torch.tanh(bias[j][1]), wires=0)
            
        qml.RZ(phi[num_layer-1]+np.pi*torch.tanh(bias[num_layer-1][2]), wires=0)
    return qml.state()

Define simple cost function, weighted fidelity function

In [230]:
# simple cost function
def cost(params,x,y, num_layer):
    los = 0
    for i in range(len(x)):
        res = compact_circ2(params,x[i], num_layer)
        fidelity = (res[1] - y[i])**(2) + (res[0]-(1-y[i]))**2
        los += fidelity
    return los / (len(x))

In [231]:
# Here, we define some functions to construct weighted fidelity cost function
def label_state(params, x, num_layer):

    res = compact_circ2(params, x, num_layer).detach().numpy()

    if res[0] >= res[1]:
        state = [1,0]
    else:
        state = [0,1]
    return state

def state_gen(params, x,y, num_layer): 
    zero_res, one_res = [], []
    zero_label, one_label = [], []
    zero_y, one_y = [], []
    for i in range(len(x)):
        if label_state(params, x[i], num_layer) == [1,0]:
            zero_res.append(compact_circ(params, x[i], num_layer))
            zero_label.append(torch.tensor([1,0]))
            y_label = torch.tensor([1-y[i], y[i]])
            zero_y.append(y_label)
        elif label_state(params, x[i], num_layer) == [0,1]:
            one_res.append(compact_circ(params, x[i], num_layer))
            one_label.append(torch.tensor([0,1]))
            y_label = torch.tensor([1-y[i], y[i]])
            one_y.append(y_label)
    if one_res == []:
        b,d,e = one_res, one_label, one_y
    else:
        b = torch.stack(one_res, dim=0)
        d = torch.stack(one_label, dim=0).type(torch.complex128)
        e = torch.stack(one_y, dim=0).type(torch.complex128)
    if zero_res == []:
        a,c,f = zero_res, zero_label, zero_y
    else:
        a = torch.stack(zero_res, dim=0)
        c = torch.stack(zero_label, dim=0).type(torch.complex128)
        f = torch.stack(zero_y, dim=0).type(torch.complex128)
    return a,b,c,d,e,f

# weighted fidelity cost fn
def weighted_fidelity_cost(params, x, y, num_layer):
            
    alpha = params[0:2]
    zero_res, one_res, zero_label, one_label, one_y, zero_y = state_gen(params, x,y, num_layer)
    weighted_fi = 0
    for j in range(len(zero_res)):
        weighted_fi += (alpha[0]*torch.inner(zero_label[j], zero_res[j])*torch.inner(zero_label[j], zero_res[j]).conj()
                        - torch.inner(zero_label[j], zero_y[j])*torch.inner(zero_label[j], zero_y[j]).conj())**2 / len(x) / 2
    for k in range(len(one_res)):
        weighted_fi += (alpha[1]*torch.inner(one_label[k], one_res[k])*torch.inner(one_label[k], one_res[k]).conj()
                        - torch.inner(one_label[k], one_y[k])*torch.inner(one_label[k], one_y[k]).conj())**2 / len(x) / 2
        
    return weighted_fi

Training with torch optimizer

In [343]:
a, b = 0, 1000
num_layer = 5
x, y = torch.Tensor(x_train_scaled[a:b]), torch.Tensor(y_train[a:b])
num_input = len(x[0])
params = torch.ones(2+(num_input+3)*num_layer, requires_grad=True)
y.mean()

tensor(0.2459)

In [13]:
# You can utilise tensorboard to see loss changing according to epochs
from torch.utils.tensorboard import SummaryWriter
import datetime
layer_name = "{} layers"
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_dir = f"dataset/{current_time}-{layer_name.format(num_layer)}/"
writer = SummaryWriter(log_dir = log_dir)

In [ ]:
# LBFGS optimizer
opt = optim.LBFGS([params], lr=1e0)
def closure():
    opt.zero_grad()
    loss = weighted_fidelity_cost(params,x,y,num_layer).type(torch.float64)
    loss.backward()
    print(loss)
    return loss
# We just simply set large epochs and stop when it is trained enough
for _ in range(10000): 
    opt.step(closure = closure)  
    # writer.add_scalar('loss', closure().type(torch.float64), _)

In [ ]:
opt = optim.Adam([params], lr=1e-1)
# We just simply set large epochs and stop when it is trained enough
for _ in range(10000):
    opt.zero_grad()
    
    loss = weighted_fidelity_cost(params,x,y,num_layer)
    # loss = cost(params,x,y,num_layer)
    loss.backward()
    opt.step()  
    # writer.add_scalar('loss', loss.type(torch.float64), _)
    print(loss)

In [345]:
# Define classify function
def classify(params,x,num_layer):
    y_hat = []
    for i in range(len(x)):
        res = compact_circ2(params, x[i], num_layer).detach().numpy()
        if res[0] >= res[1]:
            y_hat.append(0)
        else:
            y_hat.append(1)
    return y_hat

In [346]:
xt, yt = torch.Tensor(x_test_scaled), torch.Tensor(y_test)
y_hat = classify(params,xt,num_layer)
yt.mean()

tensor(0.3387)

In [347]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("accuracy : %.4f" % accuracy_score(yt,y_hat))
print("precision : %.4f" % precision_score(yt, y_hat))
print("recall : %.4f" % recall_score(yt, y_hat))
print("F1-Score : %.4f" % f1_score(yt, y_hat))

accuracy : 0.6935
precision : 0.5500
recall : 0.5238
F1-Score : 0.5366


2. Data re uploading method with Qiskit

In [27]:
import numpy as np

import qiskit
from qiskit_aer import StatevectorSimulator, AerSimulator
from qiskit import transpile, assemble, QuantumCircuit, BasicAer, IBMQ
from qiskit.visualization import *
from qiskit.circuit import Parameter, ParameterVector
from scipy.optimize import minimize
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [405]:
def QC(backend, shots, params, num_layer, x):
    
    ####### circuit ########
    circuit = QuantumCircuit(1,1)

    theta = ParameterVector('θ',num_layer*3)
    
    for i in range(num_layer):
        circuit.rz(theta[3*i],0)
        circuit.ry(theta[3*i+1],0)
        circuit.rz(theta[3*i+2],0)
    circuit.measure(0,0)

    
    ## define parameters  ##
    phi = []
    num_input = len(x[0])
    weight = params[0: num_input*num_layer].reshape(num_layer,num_input)
    bias = params[num_input*num_layer:len(params)].reshape(num_layer,3)
    for k in range(len(x)):
        for i in range(num_layer):
            for j in range(num_input):
                phi.append(np.inner(weight[i], x[k])+bias[i][j])
    phi = np.array(phi).reshape(len(x),3*num_layer).numpy()
    ####### job run ########
    circs = []
    for i in range(len(x)):
        circ = circuit.bind_parameters({theta: phi[i]})
        circs.append(circ)
    
    job = backend.run(circs)
    result = job.result().get_counts()
    prob_0, prob_1 = [], []
    for i in range(len(x)):
        if '0' in result[i]:
            prob_0.append(result[i]['0']/shots)
            prob_1.append(1 - result[i]['0']/shots)
        else:
            prob_0.append(1 - result[i]['1']/shots)
            prob_1.append(result[i]['1']/shots)

    
    return [prob_0, prob_1]

def pulse_cost(params, backend, num_layer, shots, x, y):
    result = QC(backend, shots, params, num_layer, x)
    y_hat = []
    cost = 0
    for i in range(len(x)):
        cost += (result[0][i] - y[i])**(2) + (result[1][i]-(1-y[i]))**2 
    return cost / len(x)

def pulse_classify(params, backend, num_layer, shots, xt):
    result = QC(backend, shots, params, num_layer, xt)
    y_hat = []
    for i in range(len(xt)):
        if result[0][i] > result[1][i]:
            y_hat.append(0)
        else:
            y_hat.append(1)
    return y_hat
    

In [448]:
provider = IBMQ.get_provider() # put your hub, group, project
# backend = provider.get_backend('ibm_sherbrooke')
# backend = AerSimulator.from_backend(backend)
backend = StatevectorSimulator
backend, shots, num_layer= backend, 1024, 8

In [449]:
a, b = 0, 10000
x, y = torch.Tensor(x_train_scaled), torch.Tensor(y_train)
param = np.random.random(6*num_layer).numpy()

In [450]:
out = minimize(pulse_cost, x0=param, args = (backend, num_layer, shots, x, y) ,method="L-BFGS-B", options={'maxiter':1000})

In [451]:
c,a = 0, 100000
xt, yt = torch.Tensor(x_test_scaled[c:a]), torch.Tensor(y_test[c:a])
y_hat2 = pulse_classify(out.x, backend, num_layer,shots, xt)
yt.mean()

tensor(0.3387)

In [452]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("accuracy : %.4f" % accuracy_score(yt,y_hat2))
print("precision : %.4f" % precision_score(yt, y_hat2))
print("recall : %.4f" % recall_score(yt, y_hat2))
print("F1-Score : %.4f" % f1_score(yt, y_hat2))

accuracy : 0.5323
precision : 0.3889
recall : 0.6667
F1-Score : 0.4912
